In [1]:
import sys
sys.path.append('../build')   # assume an out-of-source build

import numpy as np
import exafmm as fmm
import exafmm.helmholtz as helmholtz

In [2]:
helmholtz.__doc__

"exafmm's submodule for Helmholtz kernel"

### Create sources and targets

In [3]:
nsrcs = 100000
ntrgs = 100000

# generate random positions for particles
src_coords = np.random.random((nsrcs, 3))
trg_coords = np.random.random((nsrcs, 3))

# generate random charges for sources
src_charges = np.zeros(nsrcs, dtype=np.complex_)
src_charges.real = np.random.random(nsrcs)
src_charges.imag = np.random.random(nsrcs)

In [4]:
# create a list of source instances
sources = helmholtz.init_sources(src_coords, src_charges)

# create a list of target instances
targets = helmholtz.init_targets(trg_coords)

### Create a HelmholtzFMM instance

In [5]:
p = 10
ncrit = 400
depth = 4
wavek = 10
fmm = helmholtz.HelmholtzFMM(p, ncrit, depth, wavek)

### Setup FMM

Given sources, targets and FMM configurations, `setup()` builds the tree and interaction list and pre-compute invariant matrices.

Set `skip_P2P` to `True` to skip near-field interactions for BEM applications.

In [6]:
skip_P2P = False
tree = helmholtz.setup(sources, targets, fmm, skip_P2P)

### Evaluate

`evaluate()` triggers the evaluation and returns a $n_{trg} \times 4$ `numpy.ndarray`.
The $i$-th row of `trg_values` starts with the potential value of the $i$-th target, followed by its three gradient values.

In [7]:
trg_values = helmholtz.evaluate(tree, fmm)

P2M                  : 7.7013300e-01
M2M                  : 3.2773300e-01
P2L                  : 1.4400000e-04
M2P                  : 7.0000000e-05
P2P                  : 4.6419000e-01
M2L                  : 1.9199210e+00
L2L                  : 2.8679600e-01
L2P                  : 7.5918700e-01


In [8]:
trg_values[6]

array([ -822.4679547  -234.6779952j ,   161.3893981  -381.85256276j,
       -4350.30246562+3610.24972758j,  1273.07609646 -792.61274951j])

### Check Accuracy

`fmm.verify(tree.leafs)` returns L2-norm the relative error of potential and gradient in a list. It only works when `skip_P2P` is set to `False`.

In [9]:
fmm.verify(tree.leafs)

[2.6103128528309673e-08, 4.2883268403517903e-07]

### Update charges of sources and run FMM iteratively

In [10]:
niters = 10

for i in range(niters):
    print('-'*10 + ' iteration {} '.format(i) + '-'*10)  # print divider between iterations
    
    src_charges.real = np.random.random(nsrcs)     # generate new random charges
    src_charges.imag = np.random.random(nsrcs)          
    helmholtz.update_charges(tree, src_charges)      # update charges
    helmholtz.clear_values(tree)                     # clear values
    trg_values = helmholtz.evaluate(tree, fmm)       # evaluate potentials

    print("Error: ", fmm.verify(tree.leafs))       # check accuracy

---------- iteration 0 ----------
P2M                  : 7.8550100e-01
M2M                  : 3.9598400e-01
P2L                  : 9.0000000e-05
M2P                  : 2.1320000e-03
P2P                  : 4.7394500e-01
M2L                  : 1.8643160e+00
L2L                  : 3.0586600e-01
L2P                  : 7.4776400e-01
Error:  [2.8461308028804148e-08, 4.620627791487632e-07]
---------- iteration 1 ----------
P2M                  : 7.1312400e-01
M2M                  : 3.1176500e-01
P2L                  : 1.1500000e-04
M2P                  : 4.2000000e-05
P2P                  : 4.7704500e-01
M2L                  : 1.8861630e+00
L2L                  : 2.8140300e-01
L2P                  : 7.7438300e-01
Error:  [2.764751774113846e-08, 4.5900443804654814e-07]
---------- iteration 2 ----------
P2M                  : 7.1635600e-01
M2M                  : 2.7734400e-01
P2L                  : 1.7980000e-03
M2P                  : 2.6310000e-03
P2P                  : 4.8643800e-01
M2L      